Objective for this lab is to:

1. Use Tools (Push notification tool, custom made tools to record user contact details and quesstions that LLM cant answer and send them as push notifications)
2. Learn to deploy gradio app on huggingface spaces 

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()